In [6]:
%load_ext autoreload
%autoreload 2
# load library
import sys
import math
import io
from typing import Dict, Any
import numpy as np
import quaternion
from scipy.spatial.transform import Rotation
from habitat.core.utils import try_cv2_import
from habitat.utils.visualizations import maps
from torchlightning_utils import cli
import matplotlib.pyplot as plt
from PIL import Image
import habitat_sim

def make_simple_cfg(scene_id, seed=None):
    # simulator backend
    sim_cfg = habitat_sim.SimulatorConfiguration()
    if seed is not None:
        sim_cfg.random_seed = seed
    sim_cfg.scene_id = scene_id 

    # agent
    agent_cfg = habitat_sim.agent.AgentConfiguration()

    return habitat_sim.Configuration(sim_cfg, [agent_cfg])

In [7]:
# habitat sim

scene_id = "/datadrive/azure_storage/pactdata/habitat-data/habitat-dataset/hm3d/train/00606-W16Bm4ysK8v/W16Bm4ysK8v.basis.glb"
try:  # Needed to handle out of order cell run in Colab
    sim.close()
except NameError:
    pass
sim_cfg = make_simple_cfg(scene_id)
sim = habitat_sim.Simulator(sim_cfg)
path = habitat_sim.ShortestPath()
start = [-1.311927080154419, 2.8644237518310547, 0.7133827209472656]
end = [-3.4381978511810303, 0.06442362070083618, -5.416109085083008]
path.requested_start = start 
path.requested_end = end 
found_path = sim.pathfinder.find_path(path)
geodesic_distance = path.geodesic_distance
print(path.points)
print(geodesic_distance)
print(sim.pathfinder.is_navigable(start))
print(sim.pathfinder.is_navigable(end))

[array([-1.3119271,  2.8644238,  0.7133827], dtype=float32), array([-1.4242649 ,  2.8644238 ,  0.55743647], dtype=float32), array([-2.424265  ,  2.8644238 , -0.34256363], dtype=float32), array([-2.5242648 ,  2.8644238 , -0.44256353], dtype=float32), array([-3.1242652,  2.8644238, -1.4925637], dtype=float32), array([-3.174265 ,  2.8644238, -1.7925639], dtype=float32), array([-3.224265 ,  2.8644238, -5.692564 ], dtype=float32), array([-3.174265 ,  2.8644238, -6.042564 ], dtype=float32), array([-2.824265 ,  2.8644238, -6.042564 ], dtype=float32), array([-2.724265 ,  2.6644237, -5.692564 ], dtype=float32), array([-2.724265  ,  0.46442366, -2.0925636 ], dtype=float32), array([-2.7742648 ,  0.06442362, -1.6925635 ], dtype=float32), array([-3.0242648 ,  0.06442362, -1.6425638 ], dtype=float32), array([-3.174265  ,  0.06442362, -2.292564  ], dtype=float32), array([-3.4381979 ,  0.06442362, -5.416109  ], dtype=float32)]
17.055267333984375
True
True


In [8]:
from habitat import Agent
from typing import Union
from habitat.tasks.nav.shortest_path_follower import ShortestPathFollower
class ShortestPathFollowerAgent(Agent):
    def __init__(self, sim, goal_radius: float) -> None:
        super().__init__()
        self._agent = ShortestPathFollower(sim, goal_radius, False)

    def reset(self) -> None:
        pass

    def act(self, obs) -> dict:
        # for shortest path follower, the observation is agent's current position
        action = self._agent.get_next_action(obs)
        return {"action": action}
from habitat import Env

class SimpleEnv(Env):
    def get_reward_range(self):
        return [-1, 1]

    def get_reward(self, observations):
        return 0

log for abnormal
{'episode_id': '6', 'scene_id': '/datadrive/azure_storage/pactdata/habitat-data/habitat-dataset/hm3d/train/00606-W16Bm4ysK8v/W16Bm4ysK8v.basis.glb', 'scene_dataset_config': 'hm3d/hm3d_annotated_basis.scene_dataset_config.json', 'additional_obj_config_paths': [], 'start_position': [-1.311927080154419, 2.8644237518310547, 0.7133827209472656], 'start_rotation': [-0.0, -0.3144025504589081, -0.0, -0.9492897391319275], 'info': {'geodesic_distance': 17.055267333984375}, 'goals': [NavigationGoal(position=[-3.4381978511810303, 0.06442362070083618, -5.416109085083008], radius=0.2)], 'start_room': None, 'shortest_paths': None}

In [13]:
# reproduce results of habitat lab
from tqdm import tqdm
import habitat
env_config = habitat.get_config(config_paths="../configs/pointnav_config/pointnav_hm3d.yaml")
with habitat.config.read_write(env_config):
    env_config.habitat.dataset.split = "train"
    env_config.habitat.dataset.data_path = "/datadrive/azure_storage/pactdata/habitat-data/habitat-dataset/hm3d/debug/debug_shortest/train/train.json.gz"
    env_config.habitat.dataset.scenes_dir = "/datadrive/azure_storage/pactdata/habitat-data/habitat-dataset"
with SimpleEnv(config=env_config) as env:
    # override episodes iterator in env to skip episodes in dataset
    # simple agent from native habitat
    goal_radius = env.episodes[0].goals[0].radius
    if goal_radius is None:
        goal_radius = env_config.habitat.simulator.forward_step_size
    # agent = PACTPointNavAgent(
    #     raw_action_space=env.habitat_env.action_space, agent_config=agent_config
    # )
    agent = ShortestPathFollowerAgent(env.sim, goal_radius)
    print("Environment creation successful")

    pbar = tqdm(range(0, len(env.episodes)), desc="Episodes") 
    for episode in pbar:
        observations = env.reset()
        step = 0
        while not env.episode_over:
            # action = agent.act(observations)
            action = agent.act(env.current_episode.goals[0].position)
            # print("shortest path action: ", action)
            if action is None:
                break

            # next_observations, reward, done, info = env.step(action)
            
            next_observations = env.step(action)
            info = env.get_metrics()
            observations = next_observations

            step += 1
        print(f"{episode} takes steps {step}")

2023-01-24 20:10:33,650 Initializing dataset PointNav-v1
2023-01-24 20:10:33,659 initializing sim Sim-v0
2023-01-24 20:10:35,859 Initializing task Nav-v0


Environment creation successful


Episodes:  10%|█         | 1/10 [00:09<01:27,  9.68s/it]

0 takes steps 142


Episodes:  20%|██        | 2/10 [00:18<01:11,  8.99s/it]

...

7 takes steps 134


Episodes:  90%|█████████ | 9/10 [01:11<00:08,  8.24s/it]

8 takes steps 114


Episodes: 100%|██████████| 10/10 [01:20<00:00,  8.04s/it]

9 takes steps 120
